## Memory-Powered Agent
Memory-powered agents are advanced AI systems that incorporate long-term memory to improve their reasoning, adaptability, and contextual understanding. Unlike traditional models that rely solely on short-term prompts and retrieval mechanisms, memory-powered agents store and recall relevant past interactions, facts, or experiences. This enables them to provide more coherent, contextually aware, and personalized responses over time.

These agents utilize structured and unstructured memory storage, often integrating vector databases or specialized memory architectures like recurrent models, attention mechanisms, or embeddings. By dynamically retrieving and synthesizing information from both immediate context and past experiences, they enhance decision-making, reduce redundant queries, and maintain continuity in conversations or tasks.

A key advantage of memory-powered agents is their ability to learn and refine their responses over time. They can adapt to user preferences, remember previous conversations, and even correct past mistakes, making them highly effective in applications like customer support, virtual assistants, and AI-driven research tools. Combining retrieval-based and generative approaches, these agents push the boundaries of intelligent automation and user experience.

#### 1. Importing Dependencies


In [ ]:
import os
import json
import requests
from datetime import datetime
from typing import List, Dict, Any, Optional

#### 2. Memory-Powered Conversational Agent

This class implements a Memory-Powered Agent that enhances AI conversations by maintaining a memory of past interactions. It allows the agent to recall previous exchanges, provide context-aware responses, and generate conversation summaries.

The agent integrates OpenAI's GPT model and supports features such as adding user interactions to memory, retrieving past context, making API calls, summarizing conversations, and saving/loading memory for persistence. This enables a more personalized and coherent AI experience across multiple interactions.

In [ ]:
class MemoryPoweredAgent:
    def __init__(self, api_key: Optional[str] = None, model: str = "gpt-4o"): #initializes the agent with an API key, model selection, and memory storage
        self.api_key = api_key or os.environ.get("OPENAI_API_KEY")
        if not self.api_key:
            raise ValueError("OpenAI API key required")
        
        self.api_url = "https://api.openai.com/v1/chat/completions" # it defines the API endpoint that the 'MemoryPoweredAgent' will use to send requests to OpenAI's models
        self.model = model
        self.memory = []
    
    def add_to_memory(self, user_input: str, agent_response: str) -> None: #Saves user input and AI responses in a list
        self.memory.append({
            "timestamp": datetime.now().isoformat(),
            "user_input": user_input,
            "agent_response": agent_response
        })
    
    def get_context(self, limit: int = 5) -> str: # fetches the last limit number of conversations
        if not self.memory or limit <= 0:
            return ""
            
        recent = self.memory[-limit:] if limit else self.memory
        context = "Previous conversation:\n\n"
        for i, item in enumerate(recent):
            context += f"User: {item['user_input']}\nAgent: {item['agent_response']}\n\n"
        return context
        
    def api_request(self, messages: List[Dict[str, str]]) -> str: #Sends a chat request to OpenAI's API
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        data = {
            "model": self.model,
            "messages": messages,
            "temperature": 0.7
        }
        try:
            response = requests.post(self.api_url, headers=headers, json=data)
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
        except Exception as e:
            return f"Error: {e}"
    ##chat function
    def chat(self, user_input: str, include_memory: bool = True, limit: int = 5) -> str: #allows the user to chat with the agent while incorporating memory
        messages = [{"role": "system", "content": "You are a helpful AI assistant with memory of past conversations."}]
        
        if include_memory and self.memory:
            context = self.get_context(limit)
            messages.append({"role": "system", "content": f"Memory context: {context}"})
        
        messages.append({"role": "user", "content": user_input})
        response = self.api_request(messages)
        self.add_to_memory(user_input, response)
        return response
    
    def summarize(self) -> str: #Summarizes all stored conversations
        if not self.memory:
            return "No conversations to summarize."
        
        conversation = ""
        for item in self.memory:
            conversation += f"User: {item['user_input']}\nAgent: {item['agent_response']}\n\n"
        
        messages = [
            {"role": "system", "content": "Summarize the following conversation concisely."},
            {"role": "user", "content": f"CONVERSATION:\n{conversation}\n\nSUMMARY:"}
        ]
        
        return self.api_request(messages)
    
    def save(self, filename: str = "memory.json") -> None: #Persists memory to a file and reloads it
        with open(filename, 'w') as f:
            json.dump(self.memory, f)
    
    def load(self, filename: str = "memory.json") -> None: #Saves the conversation history in JSON format
        try:
            with open(filename, 'r') as f:
                self.memory = json.load(f)
        except FileNotFoundError:
            print(f"File {filename} not found. Starting with empty memory.")

#### 3. Example Usage of Memory-Powered Agent

In [ ]:
# Quick example
agent = MemoryPoweredAgent()  # Set API key in environment or pass as parameter

# Chat with memory
response = agent.chat("Hello! What can you help me with today?")
print("Agent:", response)

# Get another response that will remember the first interaction
response = agent.chat("Can you summarize our conversation?")
print("Agent:", response)

# Generate a summary of all interactions
summary = agent.summarize()
print("\nSummary:", summary)

# Save memory for future sessions
agent.save()

Agent: Hello! I can help you with a wide range of topics, such as answering questions, providing information, assisting with problem-solving, offering recommendations, or just having a friendly chat. What do you need help with today?
Agent: Certainly! In our previous conversation, you greeted me, and I responded by letting you know that I can assist with a variety of topics, including answering questions, providing information, solving problems, offering recommendations, or just having a friendly chat. I then asked what you needed help with.

Summary: The user greeted the agent and inquired about assistance. The agent responded by outlining the types of help available, including answering questions, providing information, solving problems, offering recommendations, or having a friendly chat, and then asked what the user needed help with.


#### 4. Interactive Demo for Memory-Powered Agent
This function provides a command-line interface to interact with the MemoryPoweredAgent. Users can chat with the agent, request conversation summaries, save and load memory, or exit the session. The agent remembers previous interactions, enabling context-aware responses for a more natural conversation experience.

The demo supports:

- Live chatting with memory retention
- Summarization of past interactions
- Saving and loading memory for persistent context
- Exit functionality to end the session

In [12]:
def interactive_demo():
    api_key = input("OpenAI API key (press Enter if set as environment variable): ")
    agent = MemoryPoweredAgent(api_key if api_key else None)
    print("Memory Agent ready. Type 'exit' to quit, 'summary' for conversation summary.")
    
    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'summary':
            print("\n===== CONVERSATION SUMMARY =====")
            print(agent.summarize())
            print("================================\n")
        elif user_input.lower() == 'save':
            filename = input("Filename to save (default: memory.json): ") or "memory.json"
            agent.save(filename)
            print(f"Saved to {filename}")
        elif user_input.lower() == 'load':
            filename = input("Filename to load (default: memory.json): ") or "memory.json"
            agent.load(filename)
            print(f"Loaded from {filename}")
        else:
            print("\nAgent:", agent.chat(user_input))

# Run the demo
interactive_demo()

Memory Agent ready. Type 'exit' to quit, 'summary' for conversation summary.


KeyboardInterrupt: 